# Tool recommendation
## (Convolutional neural network with weighted cross-entropy loss)

In [1]:
import numpy as np
import json
import warnings
import operator

import h5py
from keras.models import model_from_json
from keras import backend as K

warnings.filterwarnings("ignore")


def read_file(file_path):
    with open(file_path, 'r') as data_file:
        data = json.loads(data_file.read())
    return data


def create_model(model_path):
    reverse_dictionary = dict((str(v), k) for k, v in dictionary.items())
    model_weights = list()
    weight_ctr = 0
    for index, item in enumerate(trained_model.keys()):
        if "weight_" in item:
            d_key = "weight_" + str(weight_ctr)
            weights = trained_model.get(d_key).value
            model_weights.append(weights)
            weight_ctr += 1
    # set the model weights
    loaded_model.set_weights(model_weights)
    return loaded_model, dictionary, reverse_dictionary


def compute_recommendations(model, tool_sequence, labels, dictionary, reverse_dictionary, class_weights, topk=20, max_seq_len=25):
    tl_seq = tool_sequence.split(",")
    last_tool_name = reverse_dictionary[str(tl_seq[-1])]
    sample = np.zeros(max_seq_len)
    for idx, tool_id in enumerate(tl_seq):
        sample[idx] = int(tool_id)
    sample_reshaped = np.reshape(sample, (1, max_seq_len))
    tool_sequence_names = [reverse_dictionary[str(tool_pos)] for tool_pos in tool_sequence.split(",")]
    # predict next tools for a test path
    prediction = model.predict(sample_reshaped, verbose=0)
    weight_val = list(class_weights.values())
    weight_val = np.reshape(weight_val, (len(weight_val),))
    prediction = np.reshape(prediction, (prediction.shape[1],))
    prediction = prediction * weight_val
    prediction = prediction / float(np.max(prediction))
    prediction_pos = np.argsort(prediction, axis=-1)
    # get topk prediction
    topk_prediction_pos = prediction_pos[-topk:]
    topk_prediction_val = [int(prediction[pos] * 100) for pos in topk_prediction_pos]
    # read tool names using reverse dictionary
    pred_tool_ids = [reverse_dictionary[str(tool_pos)] for tool_pos in topk_prediction_pos]
    pred_tool_ids_sorted = dict()
    for (tool_pos, tool_pred_val) in zip(topk_prediction_pos, topk_prediction_val):
        tool_name = reverse_dictionary[str(tool_pos)]
        pred_tool_ids_sorted[tool_name] = tool_pred_val
    pred_tool_ids_sorted = dict(sorted(pred_tool_ids_sorted.items(), key=lambda kv: kv[1], reverse=True))
    ids_tools = dict()
    keys = list(pred_tool_ids_sorted.keys())
    tool_seq_name = ",".join(tool_sequence_names)
    print("Current tool sequence: ")
    print()
    print(tool_seq_name)
    print()
    print("Recommended tools for the tool sequence '%s' with their scores in decreasing order:" % tool_seq_name)
    print()
    for i in pred_tool_ids_sorted:
        print(i + "(" + str(pred_tool_ids_sorted[i]) + "%)")
    for key in pred_tool_ids_sorted:
        ids_tools[key] = dictionary[key]
    print()
    print("Tool ids:")
    print()
    for i in ids_tools:
        print(i + "(" + str(ids_tools[i]) + ")")

Using TensorFlow backend.
/home/kumara/miniconda3/envs/tool_prediction_cnn_wc/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/kumara/miniconda3/envs/tool_prediction_cnn_wc/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/kumara/miniconda3/envs/tool_prediction_cnn_wc/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dt

## Unpack trained model for prediction

In [2]:
model_path = "data/tool_recommendation_model.hdf5"
trained_model = h5py.File(model_path, 'r')
model_config = json.loads(trained_model.get('model_config').value)
dictionary = json.loads(trained_model.get('data_dictionary').value)
class_weights = json.loads(trained_model.get('class_weights').value)
loaded_model = model_from_json(model_config)

model, dictionary, reverse_dictionary = create_model(model_path)

Instructions for updating:
Colocations handled automatically by placer.


## Indices of tools

In [3]:
print(reverse_dictionary)

{'1': 'seq_filter_by_mapping', '2': 'Remove_ending', '3': 'cmv', '4': 'deeptools_correct_gc_bias', '5': 'sklearn_clf_metrics', '6': 'ggplot2_heatmap', '7': 'tp_tac', '8': 'rna_starsolo', '9': 'secretbt2test', '10': 'fasplit', '11': 'nonpareil', '12': 'stacks_populations', '13': 'ctb_chemfp_nxn_clustering', '14': 'bio3d_rmsd', '15': 'prepare_ligands_for_docking', '16': 'picard_CollectGcBiasMetrics', '17': 'humann2_regroup_table', '18': 'kraken2', '19': 'gatk2_unified_genotyper', '20': 'ctb_multi_obgrep', '21': 'deeptools_multi_bam_summary', '22': 'bedtools_groupbybed', '23': 'mothur_seq_error', '24': 'pileup_parser', '25': 'bg_uniq', '26': 'IDFileConverter', '27': 'limma_voom', '28': 'cshl_fastq_quality_boxplot', '29': 'seurat_run_pca', '30': '__ZIP_COLLECTION__', '31': 'trimmer', '32': 'cshl_princeton_fastx_barcode_splitter', '33': 'addValue', '34': 'vegan_diversity', '35': 'peakcalling_macs14', '36': 'taxonomy_krona_chart', '37': 'deeptools_bamCompare', '38': 'varscan', '39': 'picard_

## Recommended tools

In [4]:
topk = 20 # set the maximum number of recommendations
tool_seq = "950" # give tools ids in a sequence and see the recommendations. To know all the tool ids, 
                     # please print the variable 'reverse_dictionary'
compute_recommendations(model, tool_seq, "", dictionary, reverse_dictionary, class_weights, topk)

Current tool sequence: 

rna_star

Recommended tools for the tool sequence 'rna_star' with their scores in decreasing order:

fastqc(100%)
trimmomatic(77%)
bowtie2(69%)
hisat2(61%)
bwa_mem(58%)
rna_star(58%)
fastq_to_fasta_python(50%)
freebayes(48%)
tophat2(45%)
bwa(44%)
multiqc(37%)
cutadapt(35%)
tp_head_tool(32%)
spades(32%)
fastq_paired_end_joiner(28%)
snippy(28%)
trim_galore(28%)
fastq_groomer(24%)
picard_MarkDuplicates(22%)
bg_sortmerna(20%)

Tool ids:

fastqc(584)
trimmomatic(650)
bowtie2(227)
hisat2(481)
bwa_mem(769)
rna_star(950)
fastq_to_fasta_python(1237)
freebayes(985)
tophat2(383)
bwa(1059)
multiqc(1041)
cutadapt(494)
tp_head_tool(1267)
spades(75)
fastq_paired_end_joiner(1138)
snippy(1032)
trim_galore(571)
fastq_groomer(337)
picard_MarkDuplicates(503)
bg_sortmerna(1047)
